In [21]:
# AEGIS P&L Financial Model - 5 Year Projections
# Based on Operating Forecast requirements: Excel Scale Sheet, Annual Spend Graph, 5 Tier Scale Up Plan

import pandas as pd
import numpy as np
from pathlib import Path

# Load product catalog
products_df = pd.read_csv('/Users/leonardo.galebe/Desktop/WMC/aegis_products.csv')

print("Product Catalog Summary:")
print(f"Total Products: {len(products_df)}")
print(f"\nBy Category:")
print(products_df.groupby('category')['price'].agg(['count', 'sum', 'mean']).round(2))
print(f"\nBy Segment:")
print(products_df.groupby('segment')['price'].agg(['count', 'sum', 'mean']).round(2))


Product Catalog Summary:
Total Products: 18

By Category:
                               count        sum       mean
category                                                  
Dashcam                            4    1437.98     359.50
Drone Deployment System            1  600000.00  600000.00
Personnel Surveillance             5    3256.98     651.40
Security Surveillance Systems      4   18394.98    4598.74
Traffic Cameras                    4   43050.00   10762.50

By Segment:
                              count        sum      mean
segment                                                 
Commercial                        4     549.96    137.49
Commercial Home                   2     649.98    324.99
Government + Law Enforcement      9  661993.00  73554.78
Law Enforcement                   3    2947.00    982.33


In [22]:
# ============================================================================
# TIER DEFINITIONS & REVENUE PROJECTIONS
# ============================================================================

# Define 5 Tier Scale Up Plan
TIER_DEFINITIONS = {
    'Tier 1': {
        'description': 'Startup/Proof of Concept',
        'units_sold': {'Commercial': 50, 'Commercial Home': 100, 'Government + Law Enforcement': 2, 'Law Enforcement': 5},
        'infrastructure_cost': 64.32 + 35.00,  # OpenCart + Dialogflow from spreadsheet
        'monthly_ops_cost': 1000
    },
    'Tier 2': {
        'description': 'Early Growth',
        'units_sold': {'Commercial': 150, 'Commercial Home': 300, 'Government + Law Enforcement': 5, 'Law Enforcement': 15},
        'infrastructure_cost': 78.12 + 73.75 + 6577.00,  # All services from spreadsheet
        'monthly_ops_cost': 5000
    },
    'Tier 3': {
        'description': 'Expansion',
        'units_sold': {'Commercial': 400, 'Commercial Home': 800, 'Government + Law Enforcement': 15, 'Law Enforcement': 40},
        'infrastructure_cost': 10000,
        'monthly_ops_cost': 15000
    },
    'Tier 4': {
        'description': 'Scale',
        'units_sold': {'Commercial': 1000, 'Commercial Home': 2000, 'Government + Law Enforcement': 40, 'Law Enforcement': 100},
        'infrastructure_cost': 25000,
        'monthly_ops_cost': 40000
    },
    'Tier 5': {
        'description': 'Mature Operations',
        'units_sold': {'Commercial': 2500, 'Commercial Home': 5000, 'Government + Law Enforcement': 100, 'Law Enforcement': 250},
        'infrastructure_cost': 60000,
        'monthly_ops_cost': 100000
    }
}

# Annual Growth Rate (from spreadsheet: 25%)
ANNUAL_GROWTH_RATE = 0.25

# Calculate revenue by tier
def calculate_tier_revenue(tier_name, tier_data):
    """Calculate annual revenue for a given tier"""
    revenue = 0
    for segment, units in tier_data['units_sold'].items():
        segment_products = products_df[products_df['segment'] == segment]
        if len(segment_products) > 0:
            avg_price = segment_products['price'].mean()
            revenue += units * avg_price
    return revenue

# Build tier revenue table
tier_revenue = {}
for tier_name, tier_data in TIER_DEFINITIONS.items():
    tier_revenue[tier_name] = calculate_tier_revenue(tier_name, tier_data)

tier_revenue_df = pd.DataFrame({
    'Tier': list(tier_revenue.keys()),
    'Annual Revenue': [round(v, 2) for v in tier_revenue.values()],
    'Description': [TIER_DEFINITIONS[t]['description'] for t in tier_revenue.keys()],
    'Monthly Infrastructure Cost': [TIER_DEFINITIONS[t]['infrastructure_cost'] for t in tier_revenue.keys()],
    'Monthly Operations Cost': [TIER_DEFINITIONS[t]['monthly_ops_cost'] for t in tier_revenue.keys()]
})

tier_revenue_df['Annual Infrastructure Cost'] = tier_revenue_df['Monthly Infrastructure Cost'] * 12
tier_revenue_df['Annual Operations Cost'] = tier_revenue_df['Monthly Operations Cost'] * 12
tier_revenue_df['Gross Profit'] = tier_revenue_df['Annual Revenue'] - tier_revenue_df['Annual Infrastructure Cost'] - tier_revenue_df['Annual Operations Cost']
tier_revenue_df['Gross Margin %'] = (tier_revenue_df['Gross Profit'] / tier_revenue_df['Annual Revenue'] * 100).round(2)

print("\n5-Tier Scale Up Revenue Projections:")
display(tier_revenue_df)



5-Tier Scale Up Revenue Projections:


,Tier,Annual Revenue,Description,Monthly Infrastructure Cost,Monthly Operations Cost,Annual Infrastructure Cost,Annual Operations Cost,Gross Profit,Gross Margin %
0,Tier 1,191394.72,Startup/Proof of Concept,99.32,1000,1191.84,12000,178202.88,93.11
1,Tier 2,500629.39,Early Growth,6728.87,5000,80746.44,60000,359882.95,71.89
2,Tier 3,1457603.00,Expansion,10000.00,15000,120000.00,180000,1157603.00,79.42
3,Tier 4,3827894.44,Scale,25000.00,40000,300000.00,480000,3047894.44,79.62
4,Tier 5,9569736.11,Mature Operations,60000.00,100000,720000.00,1200000,7649736.11,79.94


In [23]:
# ============================================================================
# ITEMIZED COST PROJECTIONS (from spreadsheet structure)
# ============================================================================

# Infrastructure costs breakdown (from spreadsheet)
infrastructure_costs = {
    'OpenCart - Google Cloud Hosting': {
        'unit_description': 'Bitnami OpenCart Virtual Machine 1 Shared vCPU',
        'unit_cost': 4.36,
        'unit': 'per month',
        'tier_1': 52.32,
        'tier_2': 65.40
    },
    'OpenCart - Domain': {
        'unit_description': 'Website Domain Name',
        'unit_cost': 1.00,
        'unit': 'per month',
        'tier_1': 12.00,
        'tier_2': 12.72
    },
    'Dialogflow Chatbot - Text Interaction': {
        'unit_description': 'Chatbot User Interaction (Text)',
        'unit_cost': 0.007,
        'unit': 'per interaction',
        'tier_1': 35.00,
        'tier_2': 43.75
    },
    'Dialogflow Chatbot - Audio Interaction': {
        'unit_description': 'Chatbot User Interaction (Audio)',
        'unit_cost': 0.06,
        'unit': 'per minute',
        'tier_1': 0.00,
        'tier_2': 30.00
    },
    'Twitter Sentiment Analysis - Language Services': {
        'unit_description': 'Standard Web Container - Sentiment Analysis',
        'unit_cost': 1.00,
        'unit': 'per 1,000 text records',
        'tier_1': 0.00,
        'tier_2': 1.00
    },
    'Twitter Sentiment Analysis - Logic App': {
        'unit_description': '1 vCore, 3.5 GiB RAM, 250 GB Storage',
        'unit_cost': 180.00,
        'unit': 'per month',
        'tier_1': 0.00,
        'tier_2': 2160.00
    },
    'Twitter Sentiment Analysis - SQL Database': {
        'unit_description': 'Standard Series, 2 vCore, 10.2 GB Memory',
        'unit_cost': 368.00,
        'unit': 'per month',
        'tier_1': 0.00,
        'tier_2': 4416.00
    }
}

infrastructure_df = pd.DataFrame(infrastructure_costs).T
infrastructure_df = infrastructure_df.reset_index().rename(columns={'index': 'Service'})

print("\nItemized Infrastructure Cost Projections:")
display(infrastructure_df)



Itemized Infrastructure Cost Projections:


,Service,unit_description,unit_cost,unit,tier_1,tier_2
0,OpenCart - Google Cloud Hosting,Bitnami OpenCart Virtual Machine 1 Shared vCPU,4.36,per month,52.32,65.4
1,OpenCart - Domain,Website Domain Name,1.0,per month,12.0,12.72
2,Dialogflow Chatbot - Text Interaction,Chatbot User Interaction (Text),0.007,per interaction,35.0,43.75
3,Dialogflow Chatbot - Audio Interaction,Chatbot User Interaction (Audio),0.06,per minute,0.0,30.0
4,Twitter Sentiment Analysis - Language Services,Standard Web Container - Sentiment Analysis,1.0,"per 1,000 text records",0.0,1.0
5,Twitter Sentiment Analysis - Logic App,"1 vCore, 3.5 GiB RAM, 250 GB Storage",180.0,per month,0.0,2160.0
6,Twitter Sentiment Analysis - SQL Database,"Standard Series, 2 vCore, 10.2 GB Memory",368.0,per month,0.0,4416.0


In [24]:
# ============================================================================
# 5-YEAR P&L STATEMENT WITH GROWTH ASSUMPTIONS
# ============================================================================

# Assumptions
YEARS = [2025, 2026, 2027, 2028, 2029]
GROWTH_RATE = 0.25  # 25% annual growth

# Start with Tier 1 in Year 1, scale through tiers
def build_pl_statement(start_tier='Tier 1', growth_rate=0.25):
    """Build 5-year P&L statement with tier progression"""
    
    pl_data = []
    current_tier_idx = 0
    tiers = list(TIER_DEFINITIONS.keys())
    
    for year_idx, year in enumerate(YEARS):
        # Determine tier progression: Progress through all 5 tiers over 5 years
        # Map year index to tier index (ensuring we don't exceed available tiers)
        if year_idx == 0:
            current_tier_idx = 0  # Tier 1 for year 2025
        elif year_idx == 1:
            current_tier_idx = 1  # Tier 2 for year 2026
        elif year_idx == 2:
            current_tier_idx = 2  # Tier 3 for year 2027
        elif year_idx == 3:
            current_tier_idx = 3  # Tier 4 for year 2028
        else:  # year_idx == 4
            current_tier_idx = 4  # Tier 5 for year 2029
        
        # Ensure we don't exceed available tiers
        current_tier_idx = min(current_tier_idx, len(tiers) - 1)
        
        tier_name = tiers[current_tier_idx]
        tier_data = TIER_DEFINITIONS[tier_name]
        
        # Calculate revenue with growth
        base_revenue = tier_revenue[tier_name]
        growth_multiplier = (1 + growth_rate) ** year_idx
        revenue = base_revenue * growth_multiplier
        
        # Costs
        annual_infrastructure = tier_data['infrastructure_cost'] * 12
        annual_operations = tier_data['monthly_ops_cost'] * 12
        
        # Additional operating expenses (as % of revenue)
        sales_marketing = revenue * 0.15  # 15% of revenue
        r_d = revenue * 0.10  # 10% of revenue
        general_admin = revenue * 0.08  # 8% of revenue
        salaries = tier_data['monthly_ops_cost'] * 12 * 2  # 2x ops cost for salaries
        
        total_opex = annual_infrastructure + annual_operations + sales_marketing + r_d + general_admin + salaries
        
        # COGS (Cost of Goods Sold) - product costs
        cogs_rate = 0.35  # 35% of revenue for product costs
        cogs = revenue * cogs_rate
        
        # Gross Profit
        gross_profit = revenue - cogs - annual_infrastructure
        
        # Operating Income
        operating_income = gross_profit - (annual_operations + sales_marketing + r_d + general_admin + salaries)
        
        # Taxes (assume 21% corporate tax rate)
        tax_rate = 0.21
        taxes = max(0, operating_income * tax_rate)
        
        # Net Income
        net_income = operating_income - taxes
        
        pl_data.append({
            'Year': year,
            'Tier': tier_name,
            'Revenue': round(revenue, 2),
            'COGS': round(cogs, 2),
            'Infrastructure Costs': round(annual_infrastructure, 2),
            'Gross Profit': round(gross_profit, 2),
            'Gross Margin %': round((gross_profit / revenue * 100) if revenue > 0 else 0, 2),
            'Operations Cost': round(annual_operations, 2),
            'Sales & Marketing': round(sales_marketing, 2),
            'R&D': round(r_d, 2),
            'General & Admin': round(general_admin, 2),
            'Salaries': round(salaries, 2),
            'Total Operating Expenses': round(total_opex, 2),
            'Operating Income': round(operating_income, 2),
            'Taxes (21%)': round(taxes, 2),
            'Net Income': round(net_income, 2),
            'Net Margin %': round((net_income / revenue * 100) if revenue > 0 else 0, 2)
        })
    
    return pd.DataFrame(pl_data)

# Build P&L statement
pl_statement = build_pl_statement()

print("\n" + "="*80)
print("5-YEAR PROFIT & LOSS STATEMENT")
print("="*80)
print(f"\nGrowth Assumption: {GROWTH_RATE*100}% Annual Growth")
print(f"\nFull P&L Statement:")
display(pl_statement)



5-YEAR PROFIT & LOSS STATEMENT

Growth Assumption: 25.0% Annual Growth

Full P&L Statement:


,Year,Tier,Revenue,COGS,Infrastructure Costs,Gross Profit,Gross Margin %,Operations Cost,Sales & Marketing,R&D,General & Admin,Salaries,Total Operating Expenses,Operating Income,Taxes (21%),Net Income,Net Margin %
0,2025,Tier 1,191394.72,66988.15,1191.84,123214.73,64.38,12000,28709.21,19139.47,15311.58,24000,100352.10,24054.47,5051.44,19003.03,9.93
1,2026,Tier 2,625786.74,219025.36,80746.44,326014.94,52.10,60000,93868.01,62578.67,50062.94,120000,467256.06,-60494.68,0.00,-60494.68,-9.67
2,2027,Tier 3,2277504.69,797126.64,120000.00,1360378.05,59.73,180000,341625.70,227750.47,182200.38,360000,1411576.55,68801.50,14448.31,54353.18,2.39
3,2028,Tier 4,7476356.34,2616724.72,300000.00,4559631.62,60.99,480000,1121453.45,747635.63,598108.51,960000,4207197.59,652434.03,137011.15,515422.88,6.89
4,2029,Tier 5,23363613.55,8177264.74,720000.00,14466348.81,61.92,1200000,3504542.03,2336361.36,1869089.08,2400000,12029992.47,3156356.34,662834.83,2493521.51,10.67


In [25]:
# ============================================================================
# SUMMARY VIEWS & ANNUAL SPEND GRAPH DATA
# ============================================================================

# Create summary P&L (simplified view)
summary_pl = pl_statement[['Year', 'Tier', 'Revenue', 'Gross Profit', 'Operating Income', 'Net Income', 'Gross Margin %', 'Net Margin %']].copy()

print("\n" + "="*80)
print("SUMMARY P&L (Key Metrics)")
print("="*80)
display(summary_pl)

# Annual Spend Graph Data (for visualization)
annual_spend = pl_statement[['Year', 'Revenue', 'Total Operating Expenses', 'Net Income']].copy()
annual_spend['Cumulative Revenue'] = annual_spend['Revenue'].cumsum()
annual_spend['Cumulative Net Income'] = annual_spend['Net Income'].cumsum()

print("\n" + "="*80)
print("ANNUAL SPEND GRAPH DATA")
print("="*80)
display(annual_spend)

# Key Assumptions Summary
assumptions = pd.DataFrame({
    'Assumption': [
        'Annual Growth Rate',
        'COGS as % of Revenue',
        'Sales & Marketing as % of Revenue',
        'R&D as % of Revenue',
        'General & Admin as % of Revenue',
        'Corporate Tax Rate',
        'Tier Progression Schedule'
    ],
    'Value': [
        f'{GROWTH_RATE*100}%',
        '35%',
        '15%',
        '10%',
        '8%',
        '21%',
        'Tier 1 (Y1) → Tier 2 (Y2) → Tier 3 (Y3) → Tier 4 (Y4) → Tier 5 (Y5)'
    ]
})

print("\n" + "="*80)
print("KEY ASSUMPTIONS USED FOR ANNUAL GROWTH")
print("="*80)
display(assumptions)



SUMMARY P&L (Key Metrics)


,Year,Tier,Revenue,Gross Profit,Operating Income,Net Income,Gross Margin %,Net Margin %
0,2025,Tier 1,191394.72,123214.73,24054.47,19003.03,64.38,9.93
1,2026,Tier 2,625786.74,326014.94,-60494.68,-60494.68,52.10,-9.67
2,2027,Tier 3,2277504.69,1360378.05,68801.50,54353.18,59.73,2.39
3,2028,Tier 4,7476356.34,4559631.62,652434.03,515422.88,60.99,6.89
4,2029,Tier 5,23363613.55,14466348.81,3156356.34,2493521.51,61.92,10.67



ANNUAL SPEND GRAPH DATA


,Year,Revenue,Total Operating Expenses,Net Income,Cumulative Revenue,Cumulative Net Income
0,2025,191394.72,100352.10,19003.03,191394.72,19003.03
1,2026,625786.74,467256.06,-60494.68,817181.46,-41491.65
2,2027,2277504.69,1411576.55,54353.18,3094686.15,12861.53
3,2028,7476356.34,4207197.59,515422.88,10571042.49,528284.41
4,2029,23363613.55,12029992.47,2493521.51,33934656.04,3021805.92



KEY ASSUMPTIONS USED FOR ANNUAL GROWTH


,Assumption,Value
0,Annual Growth Rate,25.0%
1,COGS as % of Revenue,35%
2,Sales & Marketing as % of Revenue,15%
3,R&D as % of Revenue,10%
4,General & Admin as % of Revenue,8%
5,Corporate Tax Rate,21%
6,Tier Progression Schedule,Tier 1 (Y1) → Tier 2 (Y2) → Tier 3 (Y3) → Tier...


In [26]:
# ============================================================================
# EXPORT TO EXCEL (Multiple Sheets)
# ============================================================================

output_path = Path('/Users/leonardo.galebe/Desktop/WMC/AEGIS_PL_Statement.xlsx')

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # Main P&L Statement
    pl_statement.to_excel(writer, sheet_name='P&L Statement', index=False)
    
    # Summary P&L
    summary_pl.to_excel(writer, sheet_name='Summary P&L', index=False)
    
    # Tier Revenue Projections
    tier_revenue_df.to_excel(writer, sheet_name='5 Tier Scale Up', index=False)
    
    # Itemized Infrastructure Costs
    infrastructure_df.to_excel(writer, sheet_name='Itemized Costs', index=False)
    
    # Annual Spend Graph Data
    annual_spend.to_excel(writer, sheet_name='Annual Spend Data', index=False)
    
    # Assumptions
    assumptions.to_excel(writer, sheet_name='Assumptions', index=False)
    
    # Product Catalog Summary
    products_summary = products_df[['name', 'category', 'segment', 'price']].copy()
    products_summary.to_excel(writer, sheet_name='Product Catalog', index=False)

print(f"\n✓ P&L Statement exported to: {output_path}")

# Also export to CSV for easy access
csv_path = Path('/Users/leonardo.galebe/Desktop/WMC/AEGIS_PL_Statement.csv')
pl_statement.to_csv(csv_path, index=False)
print(f"✓ P&L Statement also exported to: {csv_path}")

print("\n" + "="*80)
print("EXPORT COMPLETE - Ready for Excel Scale Sheet & Annual Spend Graph")
print("="*80)



✓ P&L Statement exported to: /Users/leonardo.galebe/Desktop/WMC/AEGIS_PL_Statement.xlsx
✓ P&L Statement also exported to: /Users/leonardo.galebe/Desktop/WMC/AEGIS_PL_Statement.csv

EXPORT COMPLETE - Ready for Excel Scale Sheet & Annual Spend Graph


In [27]:
# ============================================================================
# FORMATTED DISPLAY - Key Financial Metrics
# ============================================================================

print("\n" + "="*100)
print("AEGIS 5-YEAR FINANCIAL PROJECTIONS - KEY METRICS")
print("="*100)

for idx, row in pl_statement.iterrows():
    print(f"\n{row['Year']} ({row['Tier']}):")
    print(f"  Revenue:              ${row['Revenue']:,.2f}")
    print(f"  Gross Profit:         ${row['Gross Profit']:,.2f} ({row['Gross Margin %']}%)")
    print(f"  Operating Income:     ${row['Operating Income']:,.2f}")
    print(f"  Net Income:           ${row['Net Income']:,.2f} ({row['Net Margin %']}%)")
    print(f"  Total OpEx:           ${row['Total Operating Expenses']:,.2f}")

print("\n" + "="*100)
print("5-YEAR TOTALS:")
print("="*100)
print(f"  Total Revenue:         ${pl_statement['Revenue'].sum():,.2f}")
print(f"  Total Gross Profit:     ${pl_statement['Gross Profit'].sum():,.2f}")
print(f"  Total Operating Income: ${pl_statement['Operating Income'].sum():,.2f}")
print(f"  Total Net Income:       ${pl_statement['Net Income'].sum():,.2f}")
print(f"  Average Gross Margin:   {pl_statement['Gross Margin %'].mean():.2f}%")
print(f"  Average Net Margin:     {pl_statement['Net Margin %'].mean():.2f}%")
print("="*100)



AEGIS 5-YEAR FINANCIAL PROJECTIONS - KEY METRICS

2025 (Tier 1):
  Revenue:              $191,394.72
  Gross Profit:         $123,214.73 (64.38%)
  Operating Income:     $24,054.47
  Net Income:           $19,003.03 (9.93%)
  Total OpEx:           $100,352.10

2026 (Tier 2):
  Revenue:              $625,786.74
  Gross Profit:         $326,014.94 (52.1%)
  Operating Income:     $-60,494.68
  Net Income:           $-60,494.68 (-9.67%)
  Total OpEx:           $467,256.06

2027 (Tier 3):
  Revenue:              $2,277,504.69
  Gross Profit:         $1,360,378.05 (59.73%)
  Operating Income:     $68,801.50
  Net Income:           $54,353.18 (2.39%)
  Total OpEx:           $1,411,576.55

2028 (Tier 4):
  Revenue:              $7,476,356.34
  Gross Profit:         $4,559,631.62 (60.99%)
  Operating Income:     $652,434.03
  Net Income:           $515,422.88 (6.89%)
  Total OpEx:           $4,207,197.59

2029 (Tier 5):
  Revenue:              $23,363,613.55
  Gross Profit:         $14,466,348

In [28]:
# COMPILED FINANCIAL DATAFRAME WITH EVERYTHING (DEBUGGING VERSION)

compiled_df = pl_statement.copy()

# Debug: Check if tiers_df exists and is defined properly
try:
    tiers_exists = 'tiers_df' in locals() or 'tiers_df' in globals()
    print(f"tiers_df defined? {tiers_exists}")
    if tiers_exists:
        print(f">> tiers_df columns: {list(tiers_df.columns)}")
    else:
        print(">> tiers_df is NOT defined.")
except Exception as e:
    print(f"Error when checking tiers_df existence: {e}")

# Add additional context columns if available and possible
try:
    if ('tiers_df' in locals() or 'tiers_df' in globals()) and hasattr(tiers_df, 'columns'):
        if 'Annual Revenue' in tiers_df.columns:
            compiled_df = compiled_df.merge(
                tiers_df[['Tier', 'Annual Revenue', 'Description']],
                left_on='Tier', right_on='Tier', how='left'
            )
            print("Merged tiers_df into compiled_df.")
        else:
            print("'Annual Revenue' not in tiers_df columns; skipping merge.")
    else:
        print("tiers_df not available for merge.")
except Exception as e:
    print(f"Error during merge with tiers_df: {e}")

# Debug: Check assumptions_df integrity
try:
    print(f">> assumptions_df columns: {list(assumptions_df.columns)}")
except Exception as e:
    print(f"assumptions_df not available or has wrong format: {e}")

# Optionally add assumptions as metadata (not as columns, just for context)
try:
    assumptions_dict = assumptions_df.set_index('Assumption')['Value'].to_dict()
except Exception as e:
    print(f"Error creating assumptions_dict: {e}")
    assumptions_dict = {}

# Show all columns for verification
with pd.option_context('display.max_columns', None):
    print("\nCompiled Financial DataFrame:")
    display(compiled_df)

tiers_df defined? False
>> tiers_df is NOT defined.
tiers_df not available for merge.
assumptions_df not available or has wrong format: name 'assumptions_df' is not defined
Error creating assumptions_dict: name 'assumptions_df' is not defined

Compiled Financial DataFrame:


,Year,Tier,Revenue,COGS,Infrastructure Costs,Gross Profit,Gross Margin %,Operations Cost,Sales & Marketing,R&D,General & Admin,Salaries,Total Operating Expenses,Operating Income,Taxes (21%),Net Income,Net Margin %
0,2025,Tier 1,191394.72,66988.15,1191.84,123214.73,64.38,12000,28709.21,19139.47,15311.58,24000,100352.10,24054.47,5051.44,19003.03,9.93
1,2026,Tier 2,625786.74,219025.36,80746.44,326014.94,52.10,60000,93868.01,62578.67,50062.94,120000,467256.06,-60494.68,0.00,-60494.68,-9.67
2,2027,Tier 3,2277504.69,797126.64,120000.00,1360378.05,59.73,180000,341625.70,227750.47,182200.38,360000,1411576.55,68801.50,14448.31,54353.18,2.39
3,2028,Tier 4,7476356.34,2616724.72,300000.00,4559631.62,60.99,480000,1121453.45,747635.63,598108.51,960000,4207197.59,652434.03,137011.15,515422.88,6.89
4,2029,Tier 5,23363613.55,8177264.74,720000.00,14466348.81,61.92,1200000,3504542.03,2336361.36,1869089.08,2400000,12029992.47,3156356.34,662834.83,2493521.51,10.67


In [ ]:
# Export P&L Statement to Excel
from pathlib import Path

excel_output_path = Path('/Users/leonardo.galebe/Desktop/WMC/AEGIS_PL_Statement.xlsx')

# Export the main P&L statement
pl_statement.to_excel(excel_output_path, sheet_name='P&L Statement', index=False)

print(f"✓ P&L Statement saved to Excel: {excel_output_path}")
print(f"✓ File contains {len(pl_statement)} rows of financial data")
print(f"✓ Columns: {', '.join(pl_statement.columns.tolist()[:5])}...")
